In [37]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedKFold, permutation_test_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [57]:
train = pd.read_csv('train.csv').fillna(99)
train.head()

,train_id,AA3,AA4,AA5,AA6,AA7,AA14,AA15,DG1,is_female,...,GN1,GN1_OTHERS,GN2,GN2_OTHERS,GN3,GN3_OTHERS,GN4,GN4_OTHERS,GN5,GN5_OTHERS
0,0,3,32,3.0,99.0,323011,3854,481,1975,1,...,99.0,99,99,99,99,99,99,99,99,99
1,1,2,26,99.0,8.0,268131,2441,344,1981,1,...,99.0,99,1,99,2,99,2,99,2,99
2,2,1,16,99.0,7.0,167581,754,143,1995,1,...,1.0,99,2,99,2,99,2,99,2,99
3,3,4,44,5.0,99.0,445071,5705,604,1980,1,...,99.0,99,2,99,2,99,99,99,99,99
4,4,4,43,99.0,6.0,436161,5645,592,1958,1,...,99.0,99,1,99,1,99,1,99,1,99


In [58]:
train.describe()

,train_id,AA3,AA4,AA5,AA6,AA7,AA14,AA15,DG1,is_female,...,LN1B,LN2_1,LN2_2,LN2_3,LN2_4,GN1,GN2,GN3,GN4,GN5
count,18255.000000,18255.000000,18255.000000,18255.000000,18255.000000,18255.000000,18255.000000,18255.000000,18255.000000,18255.000000,...,18255.000000,18255.000000,18255.000000,18255.000000,18255.000000,18255.000000,18255.000000,18255.000000,18255.000000,18255.000000
mean,9127.000000,2.371460,28.558313,69.314818,35.346973,291360.681676,8030.510600,352.038346,1978.073185,0.537113,...,2.397316,2.145330,2.151575,2.830731,2.835881,26.186250,6.763079,7.421912,8.925883,8.817365
std,5269.908918,1.130523,9.822629,44.329750,42.638267,98126.483799,22061.608061,179.744071,14.740675,0.498634,...,1.231487,1.407641,1.407021,1.594319,1.592863,41.931286,20.537281,21.752528,24.523345,24.413247
min,0.000000,1.000000,11.000000,1.000000,6.000000,111011.000000,96.000000,24.000000,1917.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,4563.500000,1.000000,21.000000,5.000000,6.000000,216071.000000,949.000000,178.000000,1969.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,9127.000000,2.000000,31.000000,99.000000,7.000000,313011.000000,2902.000000,354.000000,1981.000000,1.000000,...,2.000000,1.000000,1.000000,3.000000,3.000000,3.000000,2.000000,2.000000,2.000000,2.000000
75%,13690.500000,3.000000,34.000000,99.000000,99.000000,348181.000000,4609.000000,523.000000,1990.000000,1.000000,...,4.000000,3.000000,3.000000,4.000000,4.000000,4.000000,3.000000,3.000000,3.000000,3.000000
max,18254.000000,4.000000,44.000000,99.000000,99.000000,448051.000000,99999.000000,633.000000,2001.000000,1.000000,...,4.000000,5.000000,5.000000,5.000000,5.000000,99.000000,99.000000,99.000000,99.000000,99.000000


In [59]:
inc_col = ["DG3","DG4","DG6","DG12B_1","DG12B_2","DG12C_1","DG12C_2","DG13_1","DG13_2","DG13_3","DG14","DL0","DL1",
           "DL2","DL4_5","DL4_11","DL4_15","DL4_16","DL4_17","DL4_20","DL4_21","DL4_22","DL4_23","DL5","DL7","DL15",
           "DL26_5","DL27","DL28","G2P1_5","G2P1_6","G2P1_7","G2P1_11","G2P1_12","G2P1_15","G2P1_16","MT1A","MT2",
           "MT6","MT7","MT7A","MT9","MT18_2","MT18_3","MT18_4","MT18_5","FF1","FF2","FF14_20","FF16_12","FF16_13",
           "FF16_19","FF16_20","MM13","IFI16_1","IFI16_3","IFI24","FL1","FL3","FL4","FL6_1","FL6_2","FL7_6","FL8_3",
           "FL8_6","FL8_7","FL9A","FL9B","FL9C","FL10","FL12","FL13","FL14","FL16","FL17","FL18","FB19","FB20","FB21",
           "FB24","FB25","FB27_1","FB27_2","FB27_3","FB27_4","FB27_5","FB27_6","FB27_7","FB27_8","FB27_9","FB27_96",
           "LN1A","LN1B","LN2_1","LN2_2","LN2_3","LN2_4","GN1","GN2","GN3","GN4","GN5","RI4"]
x_cols = [x for x in train.columns if x in inc_col]

In [60]:
X_train, X_test, y_train, y_test = train_test_split(train[x_cols], train['is_female'], test_size=1/7.0)

In [61]:
rfc = RandomForestClassifier(n_estimators = 100, class_weight = 'balanced')
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [62]:
rfc_pred = rfc.predict(X_test)

In [63]:
print confusion_matrix(y_test, rfc_pred)

[[1051  139]
 [ 146 1272]]


In [64]:
print classification_report(y_test, rfc_pred)

             precision    recall  f1-score   support

          0       0.88      0.88      0.88      1190
          1       0.90      0.90      0.90      1418

avg / total       0.89      0.89      0.89      2608



In [65]:
print pd.DataFrame(zip(train[x_cols], rfc.feature_importances_ ))

           0         1
0        DG3  0.040232
1        DG4  0.019200
2        DG6  0.110600
3    DG12B_1  0.003777
4    DG12B_2  0.003705
5    DG12C_1  0.003842
6    DG12C_2  0.004277
7     DG13_1  0.002146
8     DG13_2  0.002319
9     DG13_3  0.001903
10      DG14  0.001746
11       DL0  0.127932
12       DL1  0.080726
13       DL2  0.043647
14     DL4_5  0.003045
15    DL4_11  0.000160
16    DL4_15  0.000127
17    DL4_16  0.001252
18    DL4_17  0.001553
19    DL4_20  0.000741
20    DL4_21  0.000435
21    DL4_22  0.000116
22    DL4_23  0.000354
23       DL5  0.012285
24       DL7  0.006122
25      DL15  0.008578
26    DL26_5  0.001874
27      DL27  0.004661
28      DL28  0.003857
29    G2P1_5  0.001625
..       ...       ...
72      FL14  0.005282
73      FL16  0.005812
74      FL17  0.005137
75      FL18  0.002796
76      FB19  0.006055
77      FB20  0.011316
78      FB21  0.002891
79      FB24  0.007321
80      FB25  0.002748
81    FB27_1  0.000791
82    FB27_2  0.001372
83    FB27_

In [66]:
test = pd.read_csv('test.csv').fillna(99)
test.head()

,test_id,AA3,AA4,AA5,AA6,AA7,AA14,AA15,DG1,DG3,...,GN1,GN1_OTHERS,GN2,GN2_OTHERS,GN3,GN3_OTHERS,GN4,GN4_OTHERS,GN5,GN5_OTHERS
0,0,4,41,99.0,7.0,417211,4479,535,1979,8,...,2.0,99,1,99,3,99,3,99,3,99
1,1,3,32,2.0,99.0,322011,3803,476,1993,1,...,1.0,99,1,99,1,99,1,99,1,99
2,2,3,36,5.0,99.0,365011,5610,585,1980,3,...,2.0,99,2,99,2,99,2,99,2,99
3,3,2,24,99.0,7.0,247061,2550,350,1991,3,...,2.0,99,2,99,2,99,2,99,2,99
4,4,3,35,99.0,8.0,358071,3233,400,1985,3,...,1.0,99,1,99,1,99,1,99,1,99


In [67]:
test_Xp = test[x_cols]

In [68]:
predict_prob_test = rfc.predict_proba(test_Xp)
predict_test = rfc.predict(test_Xp)

In [69]:
test['is_female'] = pd.DataFrame(predict_prob_test)[1]
test['predict'] = pd.DataFrame(predict_test)

In [70]:
test = test[['test_id', 'is_female']]

In [71]:
test.head()

,test_id,is_female
0,0,0.96
1,1,0.24
2,2,0.94
3,3,0.95
4,4,0.38


In [72]:
test.to_csv('result_20180206_b.csv')